# transactions_v2 aggregation

Implementation of the official rules in `LimDoHyeong/kkbox_transactions_aggregation.md` to build leakage-safe subscription features for the KKBox churn model.

## Approach

1. Load `transactions_v2.csv` and keep rows with `2015-01-01 <= transaction_date <= 2017-03-31`.
2. Apply the payment/cancel definitions from the design doc (payment = `actual_amount_paid > 0`, cancel = `is_cancel == 1` or `is_auto_renew == 0`).
3. Build user-level state, history, and recency features relative to `T = 2017-04-01`.
4. Store the aggregated table under `LimDoHyeong/data/processed/transactions_agg_ldh.csv`.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

DATA_DIR = Path("LimDoHyeong") / "data"
RAW_PATH = DATA_DIR / "raw" / "transactions_v2.csv"
OUTPUT_PATH = DATA_DIR / "processed" / "transactions_agg_ldh.csv"
T_DATE = pd.Timestamp("2017-04-01")
CUTOFF_DATE = T_DATE - pd.Timedelta(days=1)
START_DATE = pd.Timestamp("2015-01-01")
print(f"Reading data from {RAW_PATH} Saving aggregates to {OUTPUT_PATH}")

Reading data from LimDoHyeong/data/raw/transactions_v2.csv Saving aggregates to LimDoHyeong/data/processed/transactions_agg_ldh.csv


In [2]:
date_cols = ["transaction_date", "membership_expire_date"]
# df = pd.read_csv(RAW_PATH)
df = pd.read_csv("data/raw/transactions_v2.csv")
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%Y%m%d")
mask = (df["transaction_date"] >= START_DATE) & (df["transaction_date"] <= CUTOFF_DATE)
df = df.loc[mask].copy()
df["is_payment"] = df["actual_amount_paid"] > 0
df["is_cancel_event"] = (df["is_cancel"] == 1) | (df["is_auto_renew"] == 0)
print(f"Rows after leakage-safe cutoff: {df.shape[0]:,}")
df.head()

Rows after leakage-safe cutoff: 1,431,009


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,is_payment,is_cancel_event
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0,True,True
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0,True,False
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0,True,False
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1,True,True
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0,True,False


In [3]:
grouped = df.groupby("msno", sort=False)
features = pd.DataFrame(index=df["msno"].drop_duplicates())
features.index.name = "msno"
# features["has_transaction"] = 1

payment_dates = df.loc[df["is_payment"]].groupby("msno")["transaction_date"].max()
payment_gap = (T_DATE - payment_dates).dt.days
features["days_since_last_payment"] = payment_gap.reindex(features.index)
features["has_ever_paid"] = payment_dates.reindex(features.index).notna().astype(int)
features["days_since_last_payment"] = features["days_since_last_payment"].fillna(999).astype(int)

cancel_dates = df.loc[df["is_cancel_event"]].groupby("msno")["transaction_date"].max()
cancel_gap = (T_DATE - cancel_dates).dt.days
features["days_since_last_cancel"] = cancel_gap.reindex(features.index)
features["has_ever_cancelled"] = cancel_dates.reindex(features.index).notna().astype(int)
features["days_since_last_cancel"] = features["days_since_last_cancel"].fillna(999).astype(int)

last_tx_idx = df.groupby("msno")["transaction_date"].idxmax()
last_tx = df.loc[last_tx_idx, ["msno", "is_auto_renew", "payment_plan_days", "payment_method_id"]].set_index("msno")
features["is_auto_renew_last"] = last_tx["is_auto_renew"].reindex(features.index).fillna(0).astype(int)
features["last_plan_days"] = last_tx["payment_plan_days"].reindex(features.index).fillna(0).astype(int)
last_payment_method = last_tx["payment_method_id"].reindex(features.index).astype("Int64")
features["last_payment_method"] = last_payment_method.astype(str).replace("<NA>", "NONE")
features["is_free_user"] = (features["has_ever_paid"] == 0).astype(int)

payment_counts = df.loc[df["is_payment"]].groupby("msno").size()
features["total_payment_count"] = payment_counts.reindex(features.index).fillna(0).astype(int)

amounts = df.loc[df["is_payment"]].groupby("msno")["actual_amount_paid"].sum()
features["total_amount_paid"] = amounts.reindex(features.index).fillna(0).astype(int)

avg = features["total_amount_paid"] / features["total_payment_count"].replace(0, np.nan)
features["avg_amount_per_payment"] = avg.fillna(0)

unique_plan_count = grouped["payment_plan_days"].nunique()
features["unique_plan_count"] = unique_plan_count.reindex(features.index).fillna(0).astype(int)

plan_days_sum = grouped["payment_plan_days"].sum()
features["subscription_months_est"] = plan_days_sum.reindex(features.index).fillna(0) / 30.0

window_30_start = T_DATE - pd.Timedelta(days=30)
mask_30 = df["is_payment"] & (df["transaction_date"] > window_30_start) & (df["transaction_date"] <= T_DATE)
payment_count_last_30d = df.loc[mask_30].groupby("msno").size()
features["payment_count_last_30d"] = payment_count_last_30d.reindex(features.index).fillna(0).astype(int)

window_90_start = T_DATE - pd.Timedelta(days=90)
mask_90 = df["is_payment"] & (df["transaction_date"] > window_90_start) & (df["transaction_date"] <= T_DATE)
payment_count_last_90d = df.loc[mask_90].groupby("msno").size()
features["payment_count_last_90d"] = payment_count_last_90d.reindex(features.index).fillna(0).astype(int)

features = features.sort_index()
print(f"Feature table shape: {features.shape}")

Feature table shape: (1197050, 15)


In [5]:
features.head()

,days_since_last_payment,has_ever_paid,days_since_last_cancel,has_ever_cancelled,is_auto_renew_last,last_plan_days,last_payment_method,is_free_user,total_payment_count,total_amount_paid,avg_amount_per_payment,unique_plan_count,subscription_months_est,payment_count_last_30d,payment_count_last_90d
msno,,,,,,,,,,,,,,,
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,160,1,160,1,0,395,22,0,1,1599,1599.0,1,13.166667,0,0
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,17,1,999,0,1,30,41,0,1,99,99.0,1,1.000000,1,1
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1,1,999,0,1,30,39,0,2,298,149.0,1,2.000000,1,2
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,6,1,999,0,1,30,41,0,1,149,149.0,1,1.000000,1,1
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,17,1,999,0,1,30,41,0,1,149,149.0,1,1.000000,1,1


In [6]:
features.columns

Index(['days_since_last_payment', 'has_ever_paid', 'days_since_last_cancel',
       'has_ever_cancelled', 'is_auto_renew_last', 'last_plan_days',
       'last_payment_method', 'is_free_user', 'total_payment_count',
       'total_amount_paid', 'avg_amount_per_payment', 'unique_plan_count',
       'subscription_months_est', 'payment_count_last_30d',
       'payment_count_last_90d'],
      dtype='object')

In [7]:
output_df = features.reset_index()
# output_df.to_csv(OUTPUT_PATH, index=False)
# output_df.to_csv("data/processed/transactions_agg_ldh.csv", index=False)
# OUTPUT_PATH, output_df.shape

In [8]:
output_df.to_parquet("data/processed/transactions_aggregated.parquet", index=False)